In [1]:
using MLJ, Recommender

┌ Info: Precompiling Recommender [f1dcbcfb-0b1d-45c0-883e-2edcfc1d4c23]
└ @ Base loading.jl:1278


In [2]:
d = dataset("movielens1m")
rating, user, movie = Recommender.load(d);

In [3]:
train, test  = partition(eachindex(rating.movieid), 0.8, shuffle=true);

In [4]:
Xs = source(rating.userid)
ys = source(rating.movieid)
ws = source(rating.rating);

In [5]:
knn_model = kNNRecommender()
knn = machine(knn_model, Xs, ys, ws)

Machine{kNNRecommender} @298 trained 0 times.
  args: 
    1:	Source @048 ⏎ `AbstractArray{Count,1}`
    2:	Source @327 ⏎ `AbstractArray{Count,1}`
    3:	Source @017 ⏎ `AbstractArray{Count,1}`


In [6]:
yhat = predict(knn, Xs)

Node{Machine{kNNRecommender}} @642
  args:
    1:	Source @048
  formula:
    predict(
        Machine{kNNRecommender} @298, 
        Source @048)

In [7]:
fit!(yhat, rows=train)

┌ Info: Training Machine{kNNRecommender} @298.
└ @ MLJBase /Users/keisuke.yanagi/.julia/packages/MLJBase/5TNcr/src/machines.jl:319


Node{Machine{kNNRecommender}} @642
  args:
    1:	Source @048
  formula:
    predict(
        Machine{kNNRecommender} @298, 
        Source @048)

In [8]:
preds = yhat(rows=test[1:10000])

10000-element Array{Any,1}:
 [3435, 904, 1944, 910, 950, 1260, 1252, 965, 931, 954]
 [912, 1230, 910, 1952, 1250, 898, 1207, 1953, 1263, 903]
 [1036, 480, 457, 541, 2174, 1222, 1220, 32, 50, 2804]
 [608, 260, 1200, 1225, 1222, 1304, 1240, 2871, 111, 296]
 [260, 2028, 2571, 480, 1240, 1197, 1097, 1200, 858, 2916]
 [1270, 1208, 1234, 1196, 3363, 1198, 912, 923, 1394, 1304]
 [296, 527, 318, 2858, 593, 150, 2959, 356, 2396, 1265]
 [858, 1252, 1097, 2028, 296, 589, 1240, 2571, 3471, 1265]
 [2916, 1200, 1370, 380, 165, 1374, 47, 349, 593, 1917]
 [527, 318, 1221, 1394, 1225, 3421, 111, 1097, 223, 2395]
 [1270, 1197, 919, 1097, 1265, 1294, 2918, 1221, 3471, 1225]
 [2716, 1270, 2571, 296, 592, 318, 457, 1387, 2791, 480]
 [912, 1247, 1234, 1394, 1079, 2396, 1617, 1265, 2791, 1968]
 ⋮
 [2571, 480, 110, 377, 1610, 2762, 589, 380, 1573, 349]
 [858, 1252, 1097, 2028, 296, 589, 1240, 2571, 3471, 1265]
 [1198, 919, 1250, 1263, 1259, 3421, 1200, 1278, 1193, 1674]
 [364, 2858, 356, 50, 1198, 1270, 2321,

In [9]:
hitrate(preds, ys()[test[1:10000]])

0.1015